# 1. Estimate $P(X > c)$

In [6]:
# NumPy vs CuPy (GPU) — Efficient Importance Sampling (BMC vs EIS)
# Colab-ready script:
# - Prints GPU specs via CuPy (and nvidia-smi if available)
# - Runs BMC/EIS with NumPy (CPU) and CuPy (GPU) for c in {1,2,3,4}
# - Produces a table "Mean (SE)" and saves a LaTeX snippet


import math
import os
import subprocess
import time
from typing import Tuple, Dict, List

import numpy as np
import pandas as pd

# Try to import CuPy (GPU)
try:
    import cupy as cp
    CUPY_AVAILABLE = True
except Exception as e:
    CUPY_AVAILABLE = False
    cp = None

# --------------------------- Config ---------------------------
N = 1_000_000  # samples per c per method
Cs = [1, 2, 3, 4]
RANDOM_SEED = 12345

rng = np.random.default_rng(RANDOM_SEED)
if CUPY_AVAILABLE:
    cp.random.seed(RANDOM_SEED)

# --------------------- Utility / Math bits --------------------
def tail_prob_standard_normal(c: float) -> float:
    """P(X>c) for X~N(0,1) using erfc to avoid precision loss."""
    return 0.5 * math.erfc(c / math.sqrt(2.0))

def bmc_numpy(c: float, n: int, rng: np.random.Generator) -> Tuple[float, float, float]:
    """Basic Monte Carlo on CPU using NumPy.
    Returns (p_hat, SE_theory, elapsed_seconds)."""
    t0 = time.perf_counter()
    x = rng.standard_normal(n, dtype=np.float64)
    hits = (x > c)
    p_hat = hits.mean()
    elapsed = time.perf_counter() - t0

    p_true = tail_prob_standard_normal(c)
    se = math.sqrt(p_true * (1.0 - p_true) / n)
    return float(p_hat), float(se), float(elapsed)

def eis_numpy(c: float, n: int, rng: np.random.Generator) -> Tuple[float, float, float]:
    """Efficient IS with mu=c on CPU using NumPy.
    Y = 1{Z>c} * exp(c^2/2 - c Z), with Z~N(c,1)."""
    t0 = time.perf_counter()
    z = rng.standard_normal(n, dtype=np.float64) + c
    weights = np.exp(0.5 * c * c - c * z, dtype=np.float64)
    y = (z > c) * weights
    p_hat = float(y.mean())
    se = float(y.std(ddof=1) / math.sqrt(n))
    elapsed = time.perf_counter() - t0
    return p_hat, se, elapsed

def bmc_cupy(c: float, n: int) -> Tuple[float, float, float]:
    """BMC on GPU using CuPy. SE uses theoretical indicator variance."""
    assert CUPY_AVAILABLE
    device = cp.cuda.Device()
    start = time.perf_counter()
    x = cp.random.standard_normal(n, dtype=cp.float64)
    hits = x > c
    p_hat = hits.mean().item()
    # synchronize to get accurate wall time
    cp.cuda.Stream.null.synchronize()
    elapsed = time.perf_counter() - start

    p_true = tail_prob_standard_normal(c)
    se = math.sqrt(p_true * (1.0 - p_true) / n)
    return float(p_hat), float(se), float(elapsed)

def eis_cupy(c: float, n: int) -> Tuple[float, float, float]:
    """EIS with mu=c on GPU using CuPy.
    Returns (p_hat, SE_sample, elapsed)."""
    assert CUPY_AVAILABLE
    device = cp.cuda.Device()
    start = time.perf_counter()
    z = cp.random.standard_normal(n, dtype=cp.float64) + c
    weights = cp.exp(0.5 * c * c - c * z)
    y = (z > c).astype(cp.float64) * weights
    p_hat = y.mean().item()
    se = (y.std(ddof=1).item()) / math.sqrt(n)
    cp.cuda.Stream.null.synchronize()
    elapsed = time.perf_counter() - start
    return float(p_hat), float(se), float(elapsed)

# ---------------------- GPU environment info ------------------
def print_env_info():
    print("\n=== Runtime / GPU Info ===")
    print(f"Python: {os.sys.version.split()[0]}")
    print(f"NumPy:  {np.__version__}")
    print(f"CuPy:   {cp.__version__ if CUPY_AVAILABLE else 'not installed'}")
    if CUPY_AVAILABLE:
        try:
            ndev = cp.cuda.runtime.getDeviceCount()
        except Exception:
            ndev = 0
        print(f"CUDA devices: {ndev}")
        if ndev > 0:
            props = cp.cuda.runtime.getDeviceProperties(0)
            name = props["name"].decode() if isinstance(props["name"], (bytes, bytearray)) else props["name"]
            total_gb = props["totalGlobalMem"] / (1024**3)
            major, minor = props["major"], props["minor"]
            print(f"Device: {name}")
            print(f"Compute Capability: {major}.{minor}")
            print(f"Total VRAM: {total_gb:.2f} GB")
            # Try nvidia-smi for more details
            try:
                smi = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=False)
                if smi.returncode == 0:
                    print("\n[nvidia-smi]")
                    print(smi.stdout)
            except Exception:
                pass
        else:
            print("No CUDA device visible. GPU columns will show — .")
    else:
        print("CuPy not available. GPU columns will show — .")

print_env_info()

# -------------------------- Run sims --------------------------
rows: List[Dict] = []

for c in Cs:
    true_dp = tail_prob_standard_normal(c)

    # CPU runs (NumPy)
    bmc_mean_cpu, bmc_se_cpu, bmc_t_cpu = bmc_numpy(c, N, rng)
    eis_mean_cpu, eis_se_cpu, eis_t_cpu = eis_numpy(c, N, rng)
    cpu_time_combined = bmc_t_cpu + eis_t_cpu

    # GPU runs (CuPy), if available
    if CUPY_AVAILABLE:
        try:
            if cp.cuda.runtime.getDeviceCount() > 0:
                bmc_mean_gpu, bmc_se_gpu, bmc_t_gpu = bmc_cupy(c, N)
                eis_mean_gpu, eis_se_gpu, eis_t_gpu = eis_cupy(c, N)
                gpu_time_combined = bmc_t_gpu + eis_t_gpu
            else:
                gpu_time_combined = None
        except Exception:
            gpu_time_combined = None
    else:
        gpu_time_combined = None

    rows.append({
        "c": c,
        "True DP": true_dp,
        "BMC Mean": bmc_mean_cpu,
        "BMC SE": bmc_se_cpu,
        "EIS Mean": eis_mean_cpu,
        "EIS SE": eis_se_cpu,
        "CPU Time (s)": cpu_time_combined,
        "GPU Time (s)": gpu_time_combined
    })

df = pd.DataFrame(rows)

# Format columns for pretty display (Mean (SE))
df_display = pd.DataFrame({
    "c": df["c"],
    "True DP": df["True DP"].map(lambda x: f"{x:.6g}"),
    "BMC Mean (SE)": (df["BMC Mean"].map(lambda x: f"{x:.6g}") + " (" +
                      df["BMC SE"].map(lambda x: f"{x:.3g}") + ")"),
    "EIS Mean (SE)": (df["EIS Mean"].map(lambda x: f"{x:.6g}") + " (" +
                      df["EIS SE"].map(lambda x: f"{x:.3g}") + ")"),
    "CPU Time (s)": df["CPU Time (s)"].map(lambda x: f"{x:.3f}"),
    "GPU Time (s)": df["GPU Time (s)"].map(lambda x: "—" if x is None else f"{x:.3f}")
})

df_display


=== Runtime / GPU Info ===
Python: 3.12.11
NumPy:  2.0.2
CuPy:   13.3.0
CUDA devices: 1
Device: Tesla T4
Compute Capability: 7.5
Total VRAM: 14.74 GB

[nvidia-smi]
Sat Oct  4 08:32:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P0             28W /   70W |     138MiB /  15360MiB 

,c,True DP,BMC Mean (SE),EIS Mean (SE),CPU Time (s),GPU Time (s)
0,1,0.158655,0.158855 (0.000365),0.158891 (0.000192),0.042,0.006
1,2,0.0227501,0.022784 (0.000149),0.0227618 (3.48e-05),0.039,0.006
2,3,0.0013499,0.001324 (3.67e-05),0.00134977 (2.48e-06),0.040,0.006
3,4,3.16712e-05,3.1e-05 (5.63e-06),3.16525e-05 (6.73e-08),0.042,0.006


# 2. Estimate $CVaR$

In [8]:
# We compute:
#  - True C-VaR using the analytic formula phi(c)/barPhi(c)
#  - BMC estimate and SE (delta method for ratio estimator)
#  - IS estimate and SE under proposal N(c,1) with weight w = exp(c^2/2 - c Z)

import math
import numpy as np
import pandas as pd

def true_cvar(c: float) -> float:
    phi = (1.0 / math.sqrt(2.0 * math.pi)) * math.exp(-0.5 * c * c)
    phibar = 0.5 * math.erfc(c / math.sqrt(2.0))
    return phi / phibar

def bmc_cvar_and_se(N: int, c: float, seed: int = 42):
    rng = np.random.default_rng(seed)
    X = rng.standard_normal(size=N)
    b = (X > c).astype(np.float64)      # indicator
    a = X * b
    sum_b = b.sum()
    if sum_b == 0.0:
        return float("nan"), float("nan")
    m_hat = float(a.sum() / sum_b)
    r = a - m_hat * b                   # residuals for delta-method variance
    # sample variance with ddof=1 for robustness
    s2 = float(r.var(ddof=1))
    bbar = float(b.mean())
    se = math.sqrt(s2 / (N * (bbar ** 2)))
    return m_hat, se

def is_cvar_and_se(N: int, c: float, seed: int = 123):
    rng = np.random.default_rng(seed)
    Z = rng.standard_normal(size=N) + c  # proposal N(c,1)
    w = np.exp(0.5 * (c**2) - c * Z)     # likelihood ratio f/g
    b = (Z > c).astype(np.float64) * w
    a = Z * b
    sum_b = b.sum()
    if sum_b == 0.0:
        return float("nan"), float("nan")
    m_hat = float(a.sum() / sum_b)
    r = a - m_hat * b
    s2 = float(r.var(ddof=1))
    bbar = float(b.mean())
    se = math.sqrt(s2 / (N * (bbar ** 2)))
    return m_hat, se

# ---- run ----
N = 1_000_000
cs = [3.0, 4.0, 5.0]

rows = []
for c in cs:
    true_val = true_cvar(c)
    m_bmc, se_bmc = bmc_cvar_and_se(N, c, seed=2025 + int(c))
    m_is,  se_is  = is_cvar_and_se(N, c, seed=777 + int(c))

    bmc_mean_str = "---" if math.isnan(m_bmc) else f"{m_bmc:.4f}"
    bmc_se_str   = "---" if math.isnan(se_bmc) else f"{se_bmc:.3e}"

    rows.append({
        "c": int(c),
        "BMC C-VaR (SE)": f"{bmc_mean_str} ({bmc_se_str})",
        "True C-VaR": f"{true_val:.3f}",
        "IS C-VaR (SE)": f"{m_is:.4f} ({se_is:.3e})",
    })

df = pd.DataFrame(rows, columns=["c","BMC C-VaR (SE)","True C-VaR","IS C-VaR (SE)"])

df

,c,BMC C-VaR (SE),True C-VaR,IS C-VaR (SE)
0,3,3.2936 (7.303e-03),3.283,3.2824 (4.145e-04)
1,4,4.2244 (3.476e-02),4.226,4.2262 (3.740e-04)
2,5,--- (---),5.187,5.1864 (3.391e-04)


# 3. Estimate $E[XI(X > c)]$

In [12]:
import math, time
import numpy as np
import pandas as pd

# Try CuPy
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    HAS_CUPY = False

# ---- analytic truth for X~N(0,1) ----
def phi_scalar(c: float) -> float:
    return (1.0 / math.sqrt(2.0 * math.pi)) * math.exp(-0.5 * c * c)

# ---- NumPy (CPU) estimators ----
def bmc_cpu(N: int, c: float, seed: int):
    rng = np.random.default_rng(seed)
    t0 = time.perf_counter()
    X = rng.standard_normal(N)
    Y = X * (X > c)
    mean = float(Y.mean())
    se = float(Y.std(ddof=1)) / math.sqrt(N)
    t = time.perf_counter() - t0
    return mean, se, t

def eis_cpu(N: int, c: float, seed: int):
    rng = np.random.default_rng(seed)
    t0 = time.perf_counter()
    Z = rng.standard_normal(N) + c              # proposal g = N(c,1)
    w = np.exp(0.5 * (c**2) - c * Z)            # f/g
    Y = Z * (Z > c) * w
    mean = float(Y.mean())
    se = float(Y.std(ddof=1)) / math.sqrt(N)
    t = time.perf_counter() - t0
    return mean, se, t

# ---- CuPy (GPU) estimators ----
def bmc_gpu(N: int, c: float, seed: int):
    cp.random.seed(seed)
    t0 = time.perf_counter()
    X = cp.random.standard_normal(N)
    Y = X * (X > c)
    cp.cuda.Stream.null.synchronize()
    mean = float(Y.mean().get())
    se   = float(Y.std(ddof=1).get()) / math.sqrt(N)
    t = time.perf_counter() - t0
    return mean, se, t

def eis_gpu(N: int, c: float, seed: int):
    cp.random.seed(seed)
    t0 = time.perf_counter()
    Z = cp.random.standard_normal(N) + c
    w = cp.exp(0.5 * (c**2) - c * Z)
    Y = Z * (Z > c) * w
    cp.cuda.Stream.null.synchronize()
    mean = float(Y.mean().get())
    se   = float(Y.std(ddof=1).get()) / math.sqrt(N)
    t = time.perf_counter() - t0
    return mean, se, t

# ---- run experiment ----
N = 1_000_000
c_list = [1.0, 2.0, 3.0, 4.0]

rows = []
for j, c in enumerate(c_list):
    true_v = phi_scalar(c)

    # CPU
    bmc_m_cpu, bmc_se_cpu, bmc_t = bmc_cpu(N, c, seed=10_000 + j)
    eis_m_cpu, eis_se_cpu, eis_t = eis_cpu(N, c, seed=20_000 + j)
    cpu_time = bmc_t + eis_t

    # GPU
    if HAS_CUPY:
        bmc_m_gpu, bmc_se_gpu, bmc_tg = bmc_gpu(N, c, seed=30_000 + j)
        eis_m_gpu, eis_se_gpu, eis_tg = eis_gpu(N, c, seed=40_000 + j)
        gpu_time = bmc_tg + eis_tg
        eis_mean_se_str = f"{eis_m_gpu:.6f} ({eis_se_gpu:.6e})"  # use GPU estimates in table
    else:
        gpu_time = float("nan")
        eis_mean_se_str = f"{eis_m_cpu:.6f} ({eis_se_cpu:.6e})"  # fall back to CPU numbers if GPU not available

    rows.append({
        "c": int(c),
        "True Value": true_v,
        "BMC Mean (SE)": f"{bmc_m_cpu:.6f} ({bmc_se_cpu:.6e})",
        "EIS Mean (SE)": eis_mean_se_str,
        "CPU Time (s)": cpu_time,
        "GPU Time (s)": gpu_time,
    })

# ---- build LaTeX (booktabs) ----
def fmt_time(x):
    if isinstance(x, float) and (math.isnan(x) or math.isinf(x)):
        return "---"
    return f"{x:.3f}"

latex_lines = [
    r"\begin{table}[htbp]",
    r"\centering",
    r"\caption{$E[X \mathbf{1}\{X>c\}]$ Estimation Comparison (BMC vs.\ EIS)}",
    r"\label{tab:xi_bmc_eis}",
    r"\begin{tabular}{c c c c c c}",
    r"\toprule",
    r"$c$ & True Value & BMC Mean (SE) & EIS Mean (SE) & CPU Time (s) & GPU Time (s) \\",
    r"\midrule",
]
for r in rows:
    latex_lines.append(
        f"{r['c']} & {r['True Value']:.6f} & {r['BMC Mean (SE)']} & "
        f"{r['EIS Mean (SE)']} & {fmt_time(r['CPU Time (s)'])} & {fmt_time(r['GPU Time (s)'])} \\\\"
    )
latex_lines += [r"\bottomrule", r"\end{tabular}", r"\end{table}"]
latex_code = "\n".join(latex_lines)

# Show DataFrame for quick comparison in Colab
df = pd.DataFrame(rows, columns=["c","True Value","BMC Mean (SE)","EIS Mean (SE)","CPU Time (s)","GPU Time (s)"])
df

,c,True Value,BMC Mean (SE),EIS Mean (SE),CPU Time (s),GPU Time (s)
0,1,0.241971,0.241924 (5.845184e-04),0.242071 (2.552147e-04),0.049042,0.008566
1,2,0.053991,0.053713 (3.567679e-04),0.053971 (7.568887e-05),0.044671,0.008445
2,3,0.004432,0.004453 (1.210529e-04),0.004429 (7.769168e-06),0.049960,0.009137
3,4,1.338302e-04,0.000147 (2.479159e-05),0.000134 (2.758473e-07),0.046925,0.008501
